In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, io, torch
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModel
from PIL import Image, UnidentifiedImageError



In [25]:
ROOT = os.path.abspath(os.path.join(os.getcwd(),".."))

DATA_DIR = os.path.join(ROOT, "data")

CLASSES = [c for c in os.listdir(DATA_DIR)]

print(CLASSES)

['Aishwarya Rai', 'Hrithik Roshan', 'NTR']


In [37]:
Total_images = 0
class_counts = {}
for cls in CLASSES:
    cls_path = os.path.join(DATA_DIR, cls)
    count = 0
    for f in os.listdir(cls_path):
        Total_images += 1
        count += 1
    class_counts[cls] = count
print(Total_images)
print(class_counts)

18
{'Aishwarya Rai': 6, 'Hrithik Roshan': 6, 'NTR': 6}


In [39]:
# pretrained Vision Transformer (ViT)
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModel.from_pretrained("google/vit-base-patch16-224").eval()

print("✅ Model loaded:", model.__class__.__name__)
print("Embedding size:", model.config.hidden_size)

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded: ViTModel
Embedding size: 768


In [42]:
from PIL import Image

# pick first image of first class
sample_img = os.path.join(DATA_DIR, classes[0], os.listdir(os.path.join(DATA_DIR, classes[0]))[0])
print("Testing:", sample_img)

img = Image.open(sample_img).convert("RGB")
inputs = processor(images=img, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

emb = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
print("Embedding shape:", emb.shape)  # should be (1, 768)


Testing: c:\Users\HP\Desktop\Python_code_practice\ml_projects\face_rec\data\Aishwarya Rai\1.jpg
Embedding shape: torch.Size([1, 768])
